# 방향
 - dict로 after값을 계속 저장해서 유지하기 --> 이러면 추후에 앞번 것을 가져올 필요가 없음

In [12]:
import datetime
import requests
import json
import pandas as pd
import time

In [14]:
after_df = pd.read_excel('산악투어_API_after.xlsx')

access_token = ''
band_key = ''
locale = 'ko-KR'
headers = {'X-Naver-Client-Id':'', 'X-naver-Client-secret':''}


call_count = after_df.shape[0] - 1 # 처음시작 시 call_count는 1을 넣어라.
after = after_df['after값'].tolist()[-1]

In [15]:
df = pd.DataFrame()
t1 = time.time()
after_dict = {}



while True:
    
    # url 불러오기
    if call_count == 1:
        url = f'https://openapi.band.us/v2/band/posts.json?access_token={access_token}&band_key={band_key}&locale={locale}'
        r = requests.get(url, headers=headers, timeout = 10)
        r_json = json.loads(r.text)
        
        
    else:
        url = f'https://openapi.band.us/v2/band/posts.json?access_token={access_token}&band_key={band_key}&locale={locale}&after={after}'
        r = requests.get(url, headers=headers, timeout = 10)
        r_json = json.loads(r.text)
        
    # 사용횟수 초과 여부 확인
    if r_json['result_code'] == 1001:
        print('오늘 사용횟수를 초과하였습니다.')
        break
        
    # after 파라미터 정의 및 stopping조건 정의
    if r_json['result_data']['paging']['next_params'] == None:
        print('다음 after 파라미터가 존재하지 않습니다. == 종료')
        break
    
    after = r_json['result_data']['paging']['next_params']['after']
    after_dict[call_count*20] = after
        
    # 데이터 수집
    writers = [r_json['result_data']['items'][idx]['author']['name'] for idx in range(20)]
    contents = [r_json['result_data']['items'][idx]['content'] for idx in range(20)]
    dates = [datetime.datetime.fromtimestamp(round(r_json['result_data']['items'][idx]['created_at']/1000)).strftime('%Y-%m-%d %H:%M:%S') for idx in range(20)]
    comment_counts = [r_json['result_data']['items'][idx]['comment_count'] for idx in range(20)]
    emotion_counts = [r_json['result_data']['items'][idx]['emotion_count'] for idx in range(20)]
    
    # 데이터 프레임에 추가
    tmp = pd.DataFrame({'작성자':writers,
                         '일자':dates,
                         '댓글개수':comment_counts,
                         '감정표현개수':emotion_counts,
                         '내용':contents})
    
    df = df.append(tmp, ignore_index=True)    
    
    # log
    if df.shape[0] % 100 == 0:
        print(f'{df.shape[0]}개 데이터 수집 완료 == 경과시간: {round(time.time()-t1)}초')
        time.sleep(1)
    
    # log2
    if df.shape[0] % 2000 == 0:
        print(f'{df.shape[0]}개 데이터 수집 완료 == 10초 휴식!!')
        time.sleep(10)
    
    # call_count += 1
    call_count += 1 

100개 데이터 수집 완료 == 경과시간: 1초
200개 데이터 수집 완료 == 경과시간: 4초
300개 데이터 수집 완료 == 경과시간: 7초
400개 데이터 수집 완료 == 경과시간: 10초
500개 데이터 수집 완료 == 경과시간: 12초
600개 데이터 수집 완료 == 경과시간: 15초
700개 데이터 수집 완료 == 경과시간: 18초
800개 데이터 수집 완료 == 경과시간: 20초
900개 데이터 수집 완료 == 경과시간: 23초
1000개 데이터 수집 완료 == 경과시간: 26초
1100개 데이터 수집 완료 == 경과시간: 28초
1200개 데이터 수집 완료 == 경과시간: 31초
1300개 데이터 수집 완료 == 경과시간: 33초
1400개 데이터 수집 완료 == 경과시간: 36초
1500개 데이터 수집 완료 == 경과시간: 39초
1600개 데이터 수집 완료 == 경과시간: 41초
1700개 데이터 수집 완료 == 경과시간: 44초
1800개 데이터 수집 완료 == 경과시간: 47초
1900개 데이터 수집 완료 == 경과시간: 49초
2000개 데이터 수집 완료 == 경과시간: 52초
2000개 데이터 수집 완료 == 10초 휴식!!
2100개 데이터 수집 완료 == 경과시간: 65초
2200개 데이터 수집 완료 == 경과시간: 68초
2300개 데이터 수집 완료 == 경과시간: 71초
2400개 데이터 수집 완료 == 경과시간: 73초
2500개 데이터 수집 완료 == 경과시간: 76초
2600개 데이터 수집 완료 == 경과시간: 79초
2700개 데이터 수집 완료 == 경과시간: 81초
2800개 데이터 수집 완료 == 경과시간: 84초
2900개 데이터 수집 완료 == 경과시간: 86초
3000개 데이터 수집 완료 == 경과시간: 89초
3100개 데이터 수집 완료 == 경과시간: 92초
3200개 데이터 수집 완료 == 경과시간: 94초
3300개 데이터 수집 완료 == 경과시간: 97초
3400개 데이터 수집 완료 == 경과시간: 10

TypeError: 'NoneType' object is not subscriptable

In [17]:
pd.DataFrame(after_dict.items(), columns = ['마지막개수','after값']).to_excel('../output/산악투어_API_after2.xlsx', index=False)